# Udacity Self-Driving Car Engineer, Term 1

# Vehicle Detection Project [P5] 

##### Summary by Michael Berner, Student @ Udacity Self Driving Car NanoDegree 2018, Stuttgart, June 19th 2018

The goals / steps of this project are the following:

* Perform a Histogram of Oriented Gradients (HOG) feature extraction on a labeled training set of images and train a Linear SVM classifier
* Optionally, you can also apply a color transform and append binned color features, as well as histograms of color, to your HOG feature vector.
* Note: for those first two steps don't forget to normalize your features and randomize a selection for training and testing.
* Implement a sliding-window technique and use your trained classifier to search for vehicles in images.
* Run your pipeline on a video stream (start with the testvideo.mp4 and later implement on full projectvideo.mp4) and create a heat map of recurring detections frame by frame to reject outliers and follow detected vehicles.
* Estimate a bounding box for vehicles detected.



## Import necessary libraries

In [1]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import pickle
import cv2
from scipy.ndimage.measurements import label

from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from skimage.feature import hog

from collections import namedtuple

from collections import deque

% matplotlib inline
print('Import of libraries DONE!')

Import of libraries DONE!


## Read SVM, linear scaler and parameters from pickle file

In [2]:
with open('./svm_scaler_result.pkl', 'rb') as f:
    # The protocol version used is detected automatically, so we do not
    # have to specify it.
    data = pickle.load(f)

# Read in previously trained Support Vector Machine (SVM)
svc = data['svc']

# Read in previously fitted scaler
X_scaler = data['X_scaler']

# Read in parameters which were applied
HP = namedtuple('HP', [], verbose=False);
HP.colorspace = ['colorspace']
HP.HOG_orient = data['orient']
HP.pix_per_cell = data['pix_per_cell']
HP.cell_per_block = data['cell_per_block']
HP.hist_bins = data['hist_bins']
HP.spatial_size = data['spatial_size']
HP.block_norm = data['block_norm']
HP.transform_sqrt = data['transform_sqrt']
print("Loading of pickle'd data DONE!")

Loading of pickle'd data DONE!


## Minor helper functions 

In [3]:
# Apply threshold to heatmap
def apply_threshold(heatmap, threshold):
    # Zero out pixels below the threshold
    heatmap[heatmap <= threshold] = 0
    # Return thresholded map
    return heatmap


# Function to draw labeled boxes around detected vehicles
def draw_labeled_bboxes(img, labels):
    # Iterate through all detected cars
    for car_number in range(1, labels[1]+1):
        # Find pixels with each car_number label value
        nonzero = (labels[0] == car_number).nonzero()
        # Identify x and y values of those pixels
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        # Define a bounding box based on min/max x and y
        bbox = ((np.min(nonzerox), np.min(nonzeroy)), (np.max(nonzerox), np.max(nonzeroy)))
        # Draw the box on the image
        cv2.rectangle(img, bbox[0], bbox[1], (0,0,255), 6)
    # Return the image
    return img


# Apply color conversion from RGB to YCrCb
def color_conversion(image):
    # apply color conversion and return converted image immediately
    return cv2.cvtColor(image, cv2.COLOR_RGB2YCrCb)

print("Minor helper function preparation DONE!")

Minor helper function preparation DONE!


## Feature extraction functions

In [4]:
# Return HOG features and visualization
def get_hog_features(img, orient, pix_per_cell, cell_per_block, 
                        vis=False, feature_vec=True, block_nrm = 'L2-Hys', trans_sqrt = False):
    features = hog(img, 
                   orientations=orient,
                   pixels_per_cell=(pix_per_cell, pix_per_cell),
                   cells_per_block=(cell_per_block, cell_per_block),
                   block_norm= block_nrm,
                   transform_sqrt=trans_sqrt,
                   visualize=vis,
                   feature_vector=feature_vec)
    return features
    

# Return spatial features
def get_spatial_features(img, size=(32,32)):
    col1 = cv2.resize(img[:,:,0], size).ravel()
    col2 = cv2.resize(img[:,:,1], size).ravel()
    col3 = cv2.resize(img[:,:,2], size).ravel()
    return np.hstack((col1,col2,col3))


# Return histogram features
def get_histogram_features(img, nbins=32):
    hist_ch1 = np.histogram(img[:,:,0], bins=nbins)
    hist_ch2 = np.histogram(img[:,:,1], bins=nbins)
    hist_ch3 = np.histogram(img[:,:,2], bins=nbins)
    # Create single feature vector by concatenating channels
    return np.concatenate((hist_ch1[0], hist_ch2[0], hist_ch3[0]))

print("Feature extraction functions prepared!")

Feature extraction functions prepared!


## Actual pipeline: "Car detection" and "Image processing"

In [5]:
def detect_cars(img,HP,svc,X_scaler,scale):
    # Initialize variables
    count = 0
    img_boxes = []
    window = 64 # Original window size
    
    # Prepare heatmap variables
    heatmap = np.zeros_like(img[:,:,0])
    
    # Have it scaled appropriately to have colorspace match the png training data
    img = img.astype(np.float32)/255
    
    # Reduce search area
    img_sub = img[HP.y_lim[0]:HP.y_lim[1],:,:]
    
    # Apply color conversion from RGB to YCrCb
    img_sub = color_conversion(img_sub)
    
    # Scale image, if necessary
    if scale != 1:
        imshape = img_sub.shape
        img_s_scld = cv2.resize(img_sub, (np.int(imshape[1]/scale), np.int(imshape[0]/scale)))
        #print("Scaled sub image from shape",imshape,"to",img_s_scld.shape)
    else:
        img_s_scld = np.copy(img_sub)
    imshape = img_s_scld.shape

    # Define blocks and steps
    nxblocks = (imshape[1] // HP.pix_per_cell) - 1
    nyblocks = (imshape[0] // HP.pix_per_cell) - 1
    nfeat_per_block = HP.HOG_orient*HP.cell_per_block**2
    
    nblocks_per_window = (window // HP.pix_per_cell) - 1
    nxsteps = (nxblocks - nblocks_per_window) // HP.cells_per_step
    nysteps = (nyblocks - nblocks_per_window) // HP.cells_per_step
    
    # Calculate hog channels for the entire scaled sub image
    hog1 = get_hog_features(img_s_scld[:,:,0], HP.HOG_orient, HP.pix_per_cell, HP.cell_per_block, feature_vec=False)
    hog2 = get_hog_features(img_s_scld[:,:,1], HP.HOG_orient, HP.pix_per_cell, HP.cell_per_block, feature_vec=False)
    hog3 = get_hog_features(img_s_scld[:,:,2], HP.HOG_orient, HP.pix_per_cell, HP.cell_per_block, feature_vec=False)

    # Perform sliding window search
    for xb in range(nxsteps):
        for yb in range(nysteps):
            count += 1
            ypos = yb*HP.cells_per_step
            xpos = xb*HP.cells_per_step
            hogfeat1 = hog1[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel()
            hogfeat2 = hog2[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel()
            hogfeat3 = hog3[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel()
            hog_features = np.hstack((hogfeat1,hogfeat2,hogfeat3))
    
            xleft = xpos*HP.pix_per_cell
            ytop = ypos*HP.pix_per_cell
            
            # Extract the image patch
            subimg = cv2.resize(img_s_scld[ytop:ytop+window, xleft:xleft+window],(64,64))
            
            # Get color features
            spatial_features = get_spatial_features(subimg, size=HP.spatial_size)
            hist_features = get_histogram_features(subimg, nbins=HP.hist_bins)
            
            # Stack features, apply scaler and make a prediction
            all_features = np.hstack((hog_features,spatial_features, hist_features))
            test_features = X_scaler.transform(all_features.reshape(1,-1))
            test_prediction = svc.predict(test_features)
            
            # If a positive match was found, add value to heatmap
            if test_prediction == 1:
                xbox_left = np.int(xleft*scale)
                ytop_draw = np.int(ytop*scale)
                win_draw = np.int(window*scale)
                heatmap[ytop_draw+HP.y_lim[0]:ytop_draw+win_draw+HP.y_lim[0], xbox_left:xbox_left+win_draw] += 1
    # Return heatmap as function output
    return heatmap


def process_image(image):
    global buffer
    # Walk through 
    heatmap1 = detect_cars(image.astype(np.float32)/255,HP,svc,X_scaler,scale[0])
    heatmap2 = detect_cars(image.astype(np.float32)/255,HP,svc,X_scaler,scale[1])
    heatmap3 = detect_cars(image.astype(np.float32)/255,HP,svc,X_scaler,scale[2])
    heatmap4 = detect_cars(image.astype(np.float32)/255,HP,svc,X_scaler,scale[3])
    heatmap5 = detect_cars(image.astype(np.float32)/255,HP,svc,X_scaler,scale[4])
    heatmap = heatmap1+heatmap2+heatmap3+heatmap4+heatmap5

    # Store heatmap in double ended queue buffer, calculate averaged heatmap over 10 frames (i.e. buffer size)
    buffer.append(heatmap)
    heatmap_avg = np.average(buffer,0)
    heatmap_avg = apply_threshold(heatmap_avg, np.max(heatmap_avg)*0.2)
    heatmap_avg = apply_threshold(heatmap_avg, 3)

    # Detect labels for individual vehicles
    labels = label(heatmap_avg)
    
    return draw_labeled_bboxes(np.copy(image),labels)

print("Pipeline functions prepared!")

Pipeline functions prepared!


## Process video file and show it in browser

In [6]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML
from collections import deque

# Initialize buffer for heatmap, define it as global variable
buffer = deque(maxlen=10)
scale = [1, 1.2500, 1.500, 1.8125, 2.2188]
HP.y_lim = [400,656]
HP.cells_per_step = 2
global buffer,HP,svc,X_scaler,scale

# Process video file
test_output = 'out_final.mp4'
# clip = VideoFileClip('test_video.mp4')
clip = VideoFileClip('project_video.mp4')
test_clip = clip.fl_image(process_image)
test_clip.write_videofile(test_output, audio=False)

[MoviePy] >>>> Building video out_final.mp4
[MoviePy] Writing video out_final.mp4


100%|█████████▉| 1260/1261 [1:34:13<00:04,  4.49s/it]


[MoviePy] Done.
[MoviePy] >>>> Video ready: out_final.mp4 



In [7]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(test_output))